In [57]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

# Launch MAPDL
mapdl = launch_mapdl(loglevel="ERROR")
mapdl.clear()
mapdl.prep7()

# Material properties -- Concrete
mapdl.units("SI")  # Use SI units (meters, kilograms, seconds, Kelvin)

# Density
mapdl.mp("DENS", 1, 2400)  # Density in kg/m³

# Define temperature-dependent material properties
mapdl.mptemp(1, 100)
mapdl.mptemp(2, 150)
mapdl.mptemp(3, 200)
mapdl.mptemp(4, 300)
mapdl.mptemp(5, 400)

# Specific Heat (J/kg*K)
mapdl.mpdata("C", 1, 1, 900)
mapdl.mpdata("C", 1, 2, 950)
mapdl.mpdata("C", 1, 3, 1000)
mapdl.mpdata("C", 1, 4, 1050)
mapdl.mpdata("C", 1, 5, 1100)

# Thermal Conductivity (W/m*K)
mapdl.mpdata("KXX", 1, 1, 2.5)
mapdl.mpdata("KXX", 1, 2, 2.463)
mapdl.mpdata("KXX", 1, 3, 2.396)
mapdl.mpdata("KXX", 1, 4, 2.259)
mapdl.mpdata("KXX", 1, 5, 2.0)

# Geometry Parameters
od_media = 0.254  # Outer diameter of media (m)
od_hx = 0.0508  # Outer diameter of HX (m)
t_hx = 0.00635  # Thickness of HX (m)
id_hx = od_hx - 2 * t_hx  # Inner diameter of HX (m)
length = 0.508  # Length of the cylinder (m)
num_radial = 5              # Radial divisions
num_circ = 12               # Circumferential divisions

# 1. Create keypoints for the circles
# Center keypoint
k0 = mapdl.k("", 0, 0, 0)

# Inner circle keypoints
inner_kps = []
outer_kps = []
for i in range(num_circ):
    angle = i * (360/num_circ)
    x_inner = (id_hx/2) * np.cos(np.deg2rad(angle))
    y_inner = (id_hx/2) * np.sin(np.deg2rad(angle))
    x_outer = (od_media/2) * np.cos(np.deg2rad(angle))
    y_outer = (od_media/2) * np.sin(np.deg2rad(angle))

    inner_kps.append(mapdl.k("", x_inner, y_inner, 0))
    outer_kps.append(mapdl.k("", x_outer, y_outer, 0))

# 2. Create circular arcs
inner_arcs = []
outer_arcs = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    inner_arcs.append(mapdl.l(inner_kps[i], inner_kps[next_i]))
    outer_arcs.append(mapdl.l(outer_kps[i], outer_kps[next_i]))

# 3. Create radial lines
radial_lines = []
for i in range(num_circ):
    radial_lines.append(mapdl.l(inner_kps[i], outer_kps[i]))

# 4. Create areas for each sector
areas = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    # Lines forming the sector: inner arc, radial line, outer arc, next radial line
    lines = [
        inner_arcs[i],
        radial_lines[next_i],  # Note the order - right side first
        outer_arcs[i],
        radial_lines[i]
    ]
    areas.append(mapdl.al(*lines))

# 5. Set element type and divisions
mapdl.et(1, "PLANE55")

# Set divisions on radial lines
mapdl.lsel('S', 'LINE', '', min(radial_lines), max(radial_lines))
mapdl.lesize('ALL', '', '', num_radial, 1)  # Uniform divisions

# Set divisions on circular arcs
mapdl.lsel('S', 'LINE', '', min(inner_arcs), max(inner_arcs))
mapdl.lesize('ALL', '', '', 1, 1)  # 1 division per arc segment
mapdl.lsel('S', 'LINE', '', min(outer_arcs), max(outer_arcs))
mapdl.lesize('ALL', '', '', 1, 1)  # 1 division per arc segment

# 6. Mesh all areas
mapdl.allsel()
mapdl.mshkey(1)  # Mapped meshing
mapdl.mshape(0, "2D")  # Quad elements
mapdl.amesh("ALL")

# 7. Visualize
mapdl.eplot(vtk=True, show_edges=True, line_width=2)

[82, 87, 110]


Widget(value='<iframe src="http://localhost:49731/index.html?ui=P_0x192c6e78f20_15&reconnect=auto" class="pyvi…

In [1]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

# Launch MAPDL
mapdl = launch_mapdl(loglevel="ERROR")
mapdl.clear()
mapdl.prep7()

# Material properties -- Concrete
mapdl.units("SI")  # Use SI units (meters, kilograms, seconds, Kelvin)

# Density
mapdl.mp("DENS", 1, 2400)  # Density in kg/m³

# Define temperature-dependent material properties
mapdl.mptemp(1, 100)
mapdl.mptemp(2, 150)
mapdl.mptemp(3, 200)
mapdl.mptemp(4, 300)
mapdl.mptemp(5, 400)

# Specific Heat (J/kg*K)
mapdl.mpdata("C", 1, 1, 900)
mapdl.mpdata("C", 1, 2, 950)
mapdl.mpdata("C", 1, 3, 1000)
mapdl.mpdata("C", 1, 4, 1050)
mapdl.mpdata("C", 1, 5, 1100)

# Thermal Conductivity (W/m*K)
mapdl.mpdata("KXX", 1, 1, 2.5)
mapdl.mpdata("KXX", 1, 2, 2.463)
mapdl.mpdata("KXX", 1, 3, 2.396)
mapdl.mpdata("KXX", 1, 4, 2.259)
mapdl.mpdata("KXX", 1, 5, 2.0)

# Geometry Parameters
od_media = 0.254  # Outer diameter of media (m)
od_hx = 0.0508  # Outer diameter of HX (m)
t_hx = 0.00635  # Thickness of HX (m)
id_hx = od_hx - 2 * t_hx  # Inner diameter of HX (m)
length = 0.508  # Length of the cylinder (m)
num_radial = 5              # Radial divisions
num_circ = 12               # Circumferential divisions

# 1. Create keypoints for the circles
# Center keypoint
k0 = mapdl.k("", 0, 0, 0)

# Inner circle keypoints
inner_kps = []
outer_kps = []
for i in range(num_circ):
    angle = i * (360/num_circ)
    x_inner = (id_hx/2) * np.cos(np.deg2rad(angle))
    y_inner = (id_hx/2) * np.sin(np.deg2rad(angle))
    x_outer = (od_media/2) * np.cos(np.deg2rad(angle))
    y_outer = (od_media/2) * np.sin(np.deg2rad(angle))

    inner_kps.append(mapdl.k("", x_inner, y_inner, 0))
    outer_kps.append(mapdl.k("", x_outer, y_outer, 0))

# 2. Create circular arcs
inner_arcs = []
outer_arcs = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    inner_arcs.append(mapdl.l(inner_kps[i], inner_kps[next_i]))
    outer_arcs.append(mapdl.l(outer_kps[i], outer_kps[next_i]))

# 3. Create radial lines
radial_lines = []
for i in range(num_circ):
    radial_lines.append(mapdl.l(inner_kps[i], outer_kps[i]))

# 4. Create areas for each sector
areas = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    # Lines forming the sector: inner arc, radial line, outer arc, next radial line
    lines = [
        inner_arcs[i],
        radial_lines[next_i],  # Note the order - right side first
        outer_arcs[i],
        radial_lines[i]
    ]
    areas.append(mapdl.al(*lines))

# 5. Set element type for 3D thermal analysis
mapdl.et(1, "SOLID70")

# 6. Extrude each 2D area to create 3D volumes
volumes = []
for area in areas:
    volumes.append(mapdl.vext(area, dz=length))

# 7. Set mesh size before meshing volumes
mapdl.lsel("S", "LINE", "", min(radial_lines), max(radial_lines))
mapdl.lesize("ALL", "", "", num_radial, 1)  # Radial divisions

# Optional: axial mesh divisions along the length
mapdl.vsel("ALL")
mapdl.vmesh("ALL")  # Mesh all volumes

# 8. Visualize 3D mesh
mapdl.eplot(vtk=True, show_edges=True, line_width=1)

[82, 87, 110]


Widget(value='<iframe src="http://localhost:55042/index.html?ui=P_0x2ad7099bc80_0&reconnect=auto" class="pyvis…

In [2]:
mapdl.finish()
mapdl.slashsolu()
mapdl.ic("ALL", "TEMP", 100)
mapdl.bf("ALL", "TEMP", 100)
mapdl.time(14400)
mapdl.bf("ALL", "TEMP", 500)
# Set Up Transient Analysis
mapdl.antype(4)  # Transient thermal analysis
mapdl.trnopt("FULL")  # Full transient solution
mapdl.kbc(0)  # Smooth boundary conditions

time_steps = np.arange(0, 14401, 600)  # From 0 to 14400 seconds, with steps every 600 seconds
for t in time_steps:
    mapdl.time(t)  # Set the current time step
    mapdl.solve()  # Solve for the current time step
mapdl.finish()

MapdlExitedError: Error:
MAPDL server connection terminated unexpectedly while running:
  SOLVE,
called by:
  _send_command

Suggestions:
  MAPDL *might* have died because it executed a not-allowed command or ran out of memory.
  Check the MAPDL command output for more details.
  Open an issue on GitHub if you need assistance: https://github.com/ansys/pymapdl/issues
Error:
  failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:50052: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)
Full error:
<_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:50052: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-04-15T17:34:52.6903177+00:00", grpc

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

# Post-Processing: Extract Min, Max, and Avg Temperature at Each Time Step
mapdl.post1()

time_steps = np.arange(0, 14400 + 600, 600)  # Time steps every 10 min
temperature_data = []

for time in time_steps:
    mapdl.set(time)

    min_temp = mapdl.get("min_temp", "NODE", 0, "TEMP", "MIN")
    max_temp = mapdl.get("max_temp", "NODE", 0, "TEMP", "MAX")
    avg_temp = mapdl.get("avg_temp", "NODE", 0, "TEMP", "AVG")

    temperature_data.append([time, min_temp, max_temp, avg_temp])

# Convert Data to Pandas DataFrame
df = pd.DataFrame(temperature_data, columns=["Time (s)", "Min Temp (°C)", "Max Temp (°C)", "Avg Temp (°C)"])
print(df)  # Print Table Output

# Plot Temperature Data
plt.figure(figsize=(8, 5))
plt.plot(df["Time (s)"], df["Min Temp (°C)"], "b-", label="Min Temp", marker="o")
plt.plot(df["Time (s)"], df["Max Temp (°C)"], "r-", label="Max Temp", marker="s")
plt.plot(df["Time (s)"], df["Avg Temp (°C)"], "g-", label="Avg Temp", marker="^")

plt.xlabel("Time (s)")
plt.ylabel("Temperature (°C)")
plt.title("Transient Temperature Analysis")
plt.legend()
plt.grid(True)

# Show Plot
plt.show()

# Exit MAPDL
mapdl.exit()